# Map points onto rivers
Station/outlet locations may not neatly map onto MERIT Hydro river locations. Here we map those points onto the closest river section to find a reasonable first guess for basin delineation. These will be fine-tuned later.

In [1]:
import sys
import numpy as np
import pandas as pd
import geopandas as gpd
from pathlib import Path
import matplotlib.pyplot as plt
from shapely.geometry import Point
from shapely.ops import nearest_points
sys.path.append(str(Path().absolute().parent))
import python_cs_functions as cs

### Config handling

In [2]:
# Specify where the config file can be found
config_file = '../0_config/config.txt'

In [3]:
# Get the required info from the config file
data_path     = cs.read_from_config(config_file,'data_path')
cs_meta_path  = cs.read_from_config(config_file,'cs_basin_path')
cs_meta_name  = cs.read_from_config(config_file,'cs_meta_name')
merit_path = cs.read_from_config(config_file, 'merit_path')
merit_bas = Path(data_path) / merit_path / 'shapes' / 'basins' / 'cat_pfaf_7_8_MERIT_Hydro_v07_Basins_v01_bugfix1_hillslopes_pfaf_7_8_clean_fixed.shp'
merit_riv = Path(data_path) / merit_path / 'shapes' / 'rivers' / 'riv_pfaf_7_8_MERIT_Hydro_v07_Basins_v01_bugfix1.shp'

### Data loading

In [4]:
# CAMELS-spat metadata file
cs_meta_path = Path(data_path) / cs_meta_path
cs_meta = pd.read_csv(cs_meta_path / cs_meta_name)

In [5]:
# Shapefiles
basins_shp = gpd.read_file(merit_bas)
rivers_shp = gpd.read_file(merit_riv)

In [50]:
# Fix up the missing hillslopes
# We are missing out on a single coastal hillslope in northern canada. Chances are we can complete the procedure anyway
#arctic_hill_files = [Path(data_path) / merit_path / 'shapes' / 'basins' / 'hillslope_84_clean_fixed.shp',
#                     Path(data_path) / merit_path / 'shapes' / 'basins' / 'hillslope_85_clean_fixed.shp']
#
#arctic_hill_84 = gpd.read_file(arctic_files[0])
#arctic_hill_85 = gpd.read_file(arctic_files[1])
#
#basins_shp = pd.concat([basins_shp,arctic_hill_84,arctic_hill_85])

### Mapping

In [6]:
# Check that we know where we're adding this
c_map_lat = np.where(cs_meta.columns == 'Mapped_lat')[0][0]
c_map_lon = np.where(cs_meta.columns == 'Mapped_lon')[0][0]
assert (cs_meta.columns[c_map_lat] == 'Mapped_lat') # If these aren't true, they'll pop an error
assert (cs_meta.columns[c_map_lon] == 'Mapped_lon')

In [7]:
def map_loop(i, debug=False, plot=False):
    
    '''Does the mapping for station at index i in the CAMELS-spat metadata file'''
        
    # 1. Get the station or outlet location
    lat,lon = cs.read_delineation_coords(cs_meta,i)
    loc = Point(lon,lat) # Location as Shapely Point()
    if debug: print('Updating gauge location {}'.format(loc))
    
    # 2. Find the subbasin and river segment we want
    mask = cs.find_subbasin_containing_point(basins_shp, loc)
    idoi = mask.COMID.values
    if debug: print('Extracting COMID {}'.format(idoi))
    small_river = cs.extract_shape_subset(rivers_shp, 'COMID', idoi)
    small_river = small_river.reset_index() # start numbering at 0 so we know what to index below
    if debug: print(small_river)
    
    # 3. Map point to river
    if len(small_river) > 0: # I.e. not a coastal hillslope
        nearest_on_stream,_ = nearest_points(small_river.loc[0,'geometry'],loc)
    else:
        nearest_on_stream = Point(-999,-999)
    
    # 4. Update metadata file
    cs_meta.iat[i,c_map_lat] = nearest_on_stream.y
    cs_meta.iat[i,c_map_lon] = nearest_on_stream.x
    
    # -. Debugging plot
    if plot:
        small_basin = cs.extract_shape_subset(basins_shp, 'COMID', idoi) # basin, testing only
        fig,ax = plt.subplots(1,1,figsize=(10,10))
        small_basin.plot(ax=ax,color='grey')
        plt.plot(loc.x,loc.y,marker='o',color='red')
        if len(small_river) > 0:
            small_river.plot(ax=ax,color='blue')
            plt.plot(nearest_on_stream.x,nearest_on_stream.y,marker='o',color='green')

    return #nearest_on_stream

In [8]:
for ix,_ in cs_meta.iterrows():
    print('Running {}'.format(ix))
    map_loop(ix)

Running 0
Running 1
Running 2
Running 3
Running 4
Running 5
Running 6
Running 7
Running 8
Running 9
Running 10
Running 11
Running 12
Running 13
Running 14
Running 15
Running 16
Running 17
Running 18
Running 19
Running 20
Running 21
Running 22
Running 23
Running 24
Running 25
Running 26
Running 27
Running 28
Running 29
Running 30
Running 31
Running 32
Running 33
Running 34
Running 35
Running 36
Running 37
Running 38
Running 39
Running 40
Running 41
Running 42
Running 43
Running 44
Running 45
Running 46
Running 47
Running 48
Running 49
Running 50
Running 51
Running 52
Running 53
Running 54
Running 55
Running 56
Running 57
Running 58
Running 59
Running 60
Running 61
Running 62
Running 63
Running 64
Running 65
Running 66
Running 67
Running 68
Running 69
Running 70
Running 71
Running 72
Running 73
Running 74
Running 75
Running 76
Running 77
Running 78
Running 79
Running 80
Running 81
Running 82
Running 83
Running 84
Running 85
Running 86
Running 87
Running 88
Running 89
Running 90
Running 9

Running 692
Running 693
Running 694
Running 695
Running 696
Running 697
Running 698
Running 699
Running 700
Running 701
Running 702
Running 703
Running 704
Running 705
Running 706
Running 707
Running 708
Running 709
Running 710
Running 711
Running 712
Running 713
Running 714
Running 715
Running 716
Running 717
Running 718
Running 719
Running 720
Running 721
Running 722
Running 723
Running 724
Running 725
Running 726
Running 727
Running 728
Running 729
Running 730
Running 731
Running 732
Running 733
Running 734
Running 735
Running 736
Running 737
Running 738
Running 739
Running 740
Running 741
Running 742
Running 743
Running 744
Running 745
Running 746
Running 747
Running 748
Running 749
Running 750
Running 751
Running 752
Running 753
Running 754
Running 755
Running 756
Running 757
Running 758
Running 759
Running 760
Running 761
Running 762
Running 763
Running 764
Running 765
Running 766
Running 767
Running 768
Running 769
Running 770
Running 771
Running 772
Running 773
Running 774
Runn

Running 1341
Running 1342
Running 1343
Running 1344
Running 1345
Running 1346
Running 1347
Running 1348
Running 1349
Running 1350
Running 1351
Running 1352
Running 1353
Running 1354
Running 1355
Running 1356
Running 1357
Running 1358
Running 1359
Running 1360
Running 1361
Running 1362
Running 1363
Running 1364
Running 1365
Running 1366
Running 1367
Running 1368
Running 1369
Running 1370
Running 1371
Running 1372
Running 1373
Running 1374
Running 1375
Running 1376
Running 1377
Running 1378
Running 1379
Running 1380
Running 1381
Running 1382
Running 1383
Running 1384
Running 1385
Running 1386
Running 1387
Running 1388
Running 1389
Running 1390
Running 1391
Running 1392
Running 1393
Running 1394
Running 1395
Running 1396
Running 1397
Running 1398
Running 1399
Running 1400
Running 1401
Running 1402
Running 1403
Running 1404
Running 1405
Running 1406
Running 1407
Running 1408
Running 1409
Running 1410
Running 1411
Running 1412
Running 1413
Running 1414
Running 1415
Running 1416
Running 1417

### Save the updated metadata file

In [9]:
cs_meta.to_csv(cs_meta_path / cs_meta_name, encoding='utf-8', index=False)